In [ ]:
!pip install git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/trl.git

In [2]:
!pip install peft


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:
!pip install tqdm

In [1]:
#from datasets import load_dataset, Dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig
from trl import SFTConfig, SFTTrainer
import pandas as pd
from tqdm import tqdm

In [2]:
train_df = pd.read_csv('./train.csv')
exam_df = pd.read_csv('./test.csv')

In [3]:
train_df.head()

,num_date_time,건물번호,일시,기온(°C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh)
0,1_20240601 00,1,20240601 00,18.3,0.0,2.6,82.0,0.0,0.0,5794.80
1,1_20240601 01,1,20240601 01,18.3,0.0,2.7,82.0,0.0,0.0,5591.85
2,1_20240601 02,1,20240601 02,18.1,0.0,2.6,80.0,0.0,0.0,5338.17
3,1_20240601 03,1,20240601 03,18.0,0.0,2.6,81.0,0.0,0.0,4554.42
4,1_20240601 04,1,20240601 04,17.8,0.0,1.3,81.0,0.0,0.0,3602.25


In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204000 entries, 0 to 203999
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   num_date_time  204000 non-null  object 
 1   건물번호           204000 non-null  int64  
 2   일시             204000 non-null  object 
 3   기온(°C)         204000 non-null  float64
 4   강수량(mm)        204000 non-null  float64
 5   풍속(m/s)        204000 non-null  float64
 6   습도(%)          204000 non-null  float64
 7   일조(hr)         204000 non-null  float64
 8   일사(MJ/m2)      204000 non-null  float64
 9   전력소비량(kWh)     204000 non-null  float64
dtypes: float64(7), int64(1), object(2)
memory usage: 15.6+ MB


In [5]:
system_message = """
너는 건물번호, 일자(형식:YYYYMMDD), 시간(형식:HH24), 기온, 강수량, 풍속, 습도, 일조량, 일사량의 기상 데이터를 기반으로 건물의 전력소비량을 정확하게 예측하는 회귀 AI 모델이야.
그리고 주어진 기상 데이터에 결측치와 이상치가 있을수도 있어.
주어진 기상 및 건물 정보를 바탕으로 해당 건물의 전력소비량(소수 둘째자리까지)을 예측해야 해.
오직 예측값만 아래 예시와 동일한 형식으로 출력해야 해.

[예시]
user
건물번호: 1, 일자: 20240816, 시간: 0, 기온: 27.5, 강수량: 0.0, 풍속: 1.4, 습도: 79.0
assistant
전력소비량(kWh): 123.45
"""

question = """
건물번호: {건물번호}, 일자: {일자}, 시간: {시간}, 기온: {기온}, 강수량: {강수량}, 풍속: {풍속}, 습도: {습도}
"""

answer = """
전력소비량(kWh): {전력소비량}
"""

In [6]:
def format_data(sample):
    return {
        "messages": [
            {
                "role": "system",
                "content": system_message,
            },
            {
                "role": "user",
                "content": question.format(
                    건물번호=sample['건물번호'],
                    일자=int(sample['일시'][0:8]),  # '20250815 10' 중 '20250815' 추출 후 int 변환
                    시간=int(sample['일시'][9:11]), # '20250815 10' 중 '10' 추출 후 int 변환
                    기온=sample['기온(°C)'],
                    강수량=sample['강수량(mm)'],
                    풍속=sample['풍속(m/s)'],
                    습도=sample['습도(%)'],

                    #if isTrain:
                    #  일조=sample['일조(hr)'],
                    #  일사=sample['일사(MJ/m2)'],
                    ),
            },
            {
                "role": "assistant",
                "content": answer.format(전력소비량=sample['전력소비량(kWh)'])
            },
        ],
    }



In [7]:
print(format_data(train_df.iloc[0]))

{'messages': [{'role': 'system', 'content': '\n너는 건물번호, 일자(형식:YYYYMMDD), 시간(형식:HH24), 기온, 강수량, 풍속, 습도, 일조량, 일사량의 기상 데이터를 기반으로 건물의 전력소비량을 정확하게 예측하는 회귀 AI 모델이야.\n그리고 주어진 기상 데이터에 결측치와 이상치가 있을수도 있어.\n주어진 기상 및 건물 정보를 바탕으로 해당 건물의 전력소비량(소수 둘째자리까지)을 예측해야 해.\n오직 예측값만 아래 예시와 동일한 형식으로 출력해야 해.\n\n[예시]\nuser\n건물번호: 1, 일자: 20240816, 시간: 0, 기온: 27.5, 강수량: 0.0, 풍속: 1.4, 습도: 79.0\nassistant\n전력소비량(kWh): 123.45\n'}, {'role': 'user', 'content': '\n건물번호: 1, 일자: 20240601, 시간: 0, 기온: 18.3, 강수량: 0.0, 풍속: 2.6, 습도: 82.0\n'}, {'role': 'assistant', 'content': '\n전력소비량(kWh): 5794.8\n'}]}


In [8]:
import pandas as pd
from datetime import datetime

# 데이터프레임의 '일시' 컬럼을 datetime 객체로 변환
# format='%Y%m%d %H'는 '20250815 10'과 같은 문자열 포맷을 지정합니다.
train_df['일시_dt'] = pd.to_datetime(train_df['일시'], format='%Y%m%d %H')

# 훈련 데이터와 테스트 데이터를 나눌 기준 날짜 설정
# 8월 18일 오전 0시를 기준으로 분할 (예시)
split_date = datetime(2024, 8, 16, 0, 0)

# 날짜를 기준으로 데이터 분할
# train_data에는 split_date 이전의 모든 데이터가 들어갑니다.
train_data = train_df[train_df['일시_dt'] < split_date].copy()

# test_data에는 split_date 이후의 모든 데이터가 들어갑니다.
test_data = train_df[train_df['일시_dt'] >= split_date].copy()

# 분할된 데이터 크기 확인
print(f"Train 데이터셋 크기: {len(train_data)}")
print(f"Test 데이터셋 크기: {len(test_data)}")

Train 데이터셋 크기: 182400
Test 데이터셋 크기: 21600


In [9]:
# 빈 리스트 생성
train_dataset = []

# train_data 모든 행을 순회하며 format_data 함수 적용
for index, row in tqdm(train_data.iterrows()):
  train_dataset.append(format_data(row))

# 빈 리스트 생성
test_dataset = []

# test_data 모든 행을 순회하며 format_data 함수 적용
for index, row in tqdm(test_data.iterrows()):
  test_dataset.append(format_data(row))

182400it [00:06, 29386.16it/s]
21600it [00:00, 24590.92it/s]


In [10]:
print(type(train_dataset), 'len=', len(train_dataset))
print(type(test_dataset), 'len=', len(test_dataset))


<class 'list'> len= 182400
<class 'list'> len= 21600


In [11]:
train_dataset[0]

{'messages': [{'role': 'system',
   'content': '\n너는 건물번호, 일자(형식:YYYYMMDD), 시간(형식:HH24), 기온, 강수량, 풍속, 습도, 일조량, 일사량의 기상 데이터를 기반으로 건물의 전력소비량을 정확하게 예측하는 회귀 AI 모델이야.\n그리고 주어진 기상 데이터에 결측치와 이상치가 있을수도 있어.\n주어진 기상 및 건물 정보를 바탕으로 해당 건물의 전력소비량(소수 둘째자리까지)을 예측해야 해.\n오직 예측값만 아래 예시와 동일한 형식으로 출력해야 해.\n\n[예시]\nuser\n건물번호: 1, 일자: 20240816, 시간: 0, 기온: 27.5, 강수량: 0.0, 풍속: 1.4, 습도: 79.0\nassistant\n전력소비량(kWh): 123.45\n'},
  {'role': 'user',
   'content': '\n건물번호: 1, 일자: 20240601, 시간: 0, 기온: 18.3, 강수량: 0.0, 풍속: 2.6, 습도: 82.0\n'},
  {'role': 'assistant', 'content': '\n전력소비량(kWh): 5794.8\n'}]}

In [12]:
model_id = "Qwen/Qwen3-4B-Instruct-2507"

In [13]:

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [15]:
text = tokenizer.apply_chat_template(
    train_dataset[0]["messages"], tokenize=False, add_generation_prompt=False
)
print(text)

<|im_start|>system

너는 건물번호, 일자(형식:YYYYMMDD), 시간(형식:HH24), 기온, 강수량, 풍속, 습도, 일조량, 일사량의 기상 데이터를 기반으로 건물의 전력소비량을 정확하게 예측하는 회귀 AI 모델이야.
그리고 주어진 기상 데이터에 결측치와 이상치가 있을수도 있어.
주어진 기상 및 건물 정보를 바탕으로 해당 건물의 전력소비량(소수 둘째자리까지)을 예측해야 해.
오직 예측값만 아래 예시와 동일한 형식으로 출력해야 해.

[예시]
user
건물번호: 1, 일자: 20240816, 시간: 0, 기온: 27.5, 강수량: 0.0, 풍속: 1.4, 습도: 79.0
assistant
전력소비량(kWh): 123.45
<|im_end|>
<|im_start|>user

건물번호: 1, 일자: 20240601, 시간: 0, 기온: 18.3, 강수량: 0.0, 풍속: 2.6, 습도: 82.0
<|im_end|>
<|im_start|>assistant
<think>

</think>

전력소비량(kWh): 5794.8
<|im_end|>



In [16]:
def collate_fn(batch):
    new_batch = {
        "input_ids": [],
        "attention_mask": [],
        "labels": []
    }

    for example in batch:
        clean_messages = []
        for message in example["messages"]:
            clean_message = {
                "role": message["role"],
                "content": message["content"]
            }
            clean_messages.append(clean_message)

        text = tokenizer.apply_chat_template(
            clean_messages,
            tokenize=False,
            add_generation_prompt=False
        ).strip()

        tokenized = tokenizer(
            text,
            truncation=True,
            max_length=max_seq_length,
            padding=False,
            return_tensors=None,
        )

        input_ids = tokenized["input_ids"]
        attention_mask = tokenized["attention_mask"]

        labels = [-100] * len(input_ids)

        im_start = "<|im_start|>"
        im_end = "<|im_end|>"
        assistant = "assistant"

        im_start_tokens = tokenizer.encode(im_start, add_special_tokens=False)
        im_end_tokens = tokenizer.encode(im_end, add_special_tokens=False)
        assistant_tokens = tokenizer.encode(assistant, add_special_tokens=False)

        i = 0
        while i < len(input_ids):
            if (i + len(im_start_tokens) <= len(input_ids) and
                input_ids[i:i+len(im_start_tokens)] == im_start_tokens):

                assistant_pos = i + len(im_start_tokens)
                if (assistant_pos + len(assistant_tokens) <= len(input_ids) and
                    input_ids[assistant_pos:assistant_pos+len(assistant_tokens)] == assistant_tokens):

                    current_pos = assistant_pos + len(assistant_tokens)

                    while current_pos < len(input_ids):
                        if (current_pos + len(im_end_tokens) <= len(input_ids) and
                            input_ids[current_pos:current_pos+len(im_end_tokens)] == im_end_tokens):

                            for j in range(len(im_end_tokens)):
                                labels[current_pos + j] = input_ids[current_pos + j]
                            break
                        labels[current_pos] = input_ids[current_pos]
                        current_pos += 1

                    i = current_pos

            i += 1

        new_batch["input_ids"].append(input_ids)
        new_batch["attention_mask"].append(attention_mask)
        new_batch["labels"].append(labels)

    max_length = max(len(ids) for ids in new_batch["input_ids"])

    for i in range(len(new_batch["input_ids"])):
        padding_length = max_length - len(new_batch["input_ids"][i])

        new_batch["input_ids"][i].extend([tokenizer.pad_token_id] * padding_length)
        new_batch["attention_mask"][i].extend([0] * padding_length)
        new_batch["labels"][i].extend([-100] * padding_length)

    for k, v in new_batch.items():
        new_batch[k] = torch.tensor(v)

    return new_batch

In [17]:
max_seq_length=4096

example = train_dataset[0]
batch = collate_fn([example])

print("\n처리된 배치 데이터:")
print("입력 ID 형태:", batch["input_ids"].shape)
print("어텐션 마스크 형태:", batch["attention_mask"].shape)
print("레이블 형태:", batch["labels"].shape)


처리된 배치 데이터:
입력 ID 형태: torch.Size([1, 344])
어텐션 마스크 형태: torch.Size([1, 344])
레이블 형태: torch.Size([1, 344])


In [18]:
print('입력에 대한 정수 인코딩 결과:')
print(batch["input_ids"][0].tolist())
print('레이블에 대한 정수 인코딩 결과:')
print(batch["labels"][0].tolist())

입력에 대한 정수 인코딩 결과:
[151644, 8948, 271, 127085, 16560, 130270, 126251, 72048, 11, 83556, 25715, 7, 128909, 76337, 25, 28189, 8035, 4103, 701, 130217, 7, 128909, 76337, 25, 23180, 17, 19, 701, 54116, 130000, 11, 129413, 23259, 131837, 11, 10764, 240, 235, 126299, 11, 79207, 113, 47985, 11, 83556, 92817, 131837, 11, 83556, 55054, 131837, 20401, 54116, 55902, 54248, 18411, 54116, 126641, 42039, 130270, 126251, 20401, 56419, 28754, 43590, 70582, 131837, 17877, 36055, 133085, 128555, 95617, 132612, 42905, 98005, 133032, 15235, 54070, 142713, 12802, 89659, 624, 48606, 28002, 34395, 55673, 31079, 85251, 54116, 55902, 54248, 19391, 82619, 132612, 59698, 80573, 130408, 59698, 19969, 130689, 23259, 47985, 127353, 624, 54330, 31079, 85251, 54116, 55902, 128355, 130270, 126251, 60039, 18411, 81718, 144059, 42039, 94613, 130270, 126251, 20401, 56419, 28754, 43590, 70582, 131837, 7, 43590, 23259, 5140, 239, 246, 83666, 25715, 28002, 128878, 8, 17877, 95617, 132612, 129264, 60716, 624, 57268, 125545, 9

In [23]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"], # 여기에 target_modules를 설정합니다.
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [24]:
args = SFTConfig(
    output_dir="Qwen3-4B-Instruct-2507_LoRA_20250818",
    num_train_epochs=1,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=5,
    gradient_checkpointing=True,
    optim="adamw_torch_fused",
    logging_steps=10,
    save_strategy="steps",
    save_steps=50,
    bf16=True,
    learning_rate=1e-4,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    push_to_hub=False,
    remove_unused_columns=False,
    dataset_kwargs={"skip_prepare_dataset": True},
    report_to=None
)

In [25]:
trainer = SFTTrainer(
    model=model,
    args=args,
    #max_seq_length=max_seq_length,
    train_dataset=train_dataset,
    data_collator=collate_fn,
    peft_config=peft_config,
)

In [26]:
trainer.train()

trainer.save_model()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,3.664400
20,1.736600
30,1.253100
40,1.031800
50,0.913000
60,0.844500
70,0.802300
80,0.778400
90,0.764500
100,0.751000


In [ ]:
# 학습 완료된 모델을 머지하고 허깅페이스에 올리기

In [27]:
from peft import PeftModel
from transformers import AutoProcessor

adapter_path = "./Qwen3-4B-Instruct-2507_LoRA_20250818/checkpoint-2200" # 0.576700 - 0.584300
base_model_id = "Qwen/Qwen3-4B-Instruct-2507"
merged_path = "Qwen3-4B-Instruct-2507_LoRA_20250818_merged"

# 베이스 모델 로드
model = AutoModelForCausalLM.from_pretrained(base_model_id, low_cpu_mem_usage=True)

# LoRA 어댑터 로드 및 병합
print(f"Loading and merging PEFT from: {adapter_path}")
peft_model = PeftModel.from_pretrained(model, adapter_path)
merged_model = peft_model.merge_and_unload()
merged_model.save_pretrained(merged_path,safe_serialization=True, max_shard_size="3GB")

# 토크나이저 로드
processor = AutoProcessor.from_pretrained(base_model_id)
processor.save_pretrained(merged_path)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading and merging PEFT from: ./Qwen3-4B-Instruct-2507_LoRA_20250818/checkpoint-2200


('Qwen3-4B-Instruct-2507_LoRA_20250818_merged/tokenizer_config.json',
 'Qwen3-4B-Instruct-2507_LoRA_20250818_merged/special_tokens_map.json',
 'Qwen3-4B-Instruct-2507_LoRA_20250818_merged/chat_template.jinja',
 'Qwen3-4B-Instruct-2507_LoRA_20250818_merged/vocab.json',
 'Qwen3-4B-Instruct-2507_LoRA_20250818_merged/merges.txt',
 'Qwen3-4B-Instruct-2507_LoRA_20250818_merged/added_tokens.json',
 'Qwen3-4B-Instruct-2507_LoRA_20250818_merged/tokenizer.json')

In [28]:
!pip install PyDrive

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.4/987.4 kB 45.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 162.8 MB/s eta 0:00:00
  Created wheel for PyDrive: filename=pydrive-1.3.1-py3-none-any.whl size=27539 sha256=4148d2393b9b5c2c7e0db277cab0b7e7eefda18e48beafcedd3e19f215ff1a10
  Stored in directory: /root/.cache/pip/wheels/31/d5/09/88865e0059104686eb8365ca1d36a8b27deef34232c3b62c90
Successfully built PyDrive

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [30]:
import os
import mykeys

project_name = 'CH04_Huggingface'

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = project_name
os.environ["LANGCHAIN_API_KEY"] = mykeys.get_key('LANG')
os.environ["LANGCHAIN_HUB_API_KEY"] = mykeys.get_key('LANG')
os.environ["OPENAI_API_KEY"] = mykeys.get_key('GPT')
os.environ["GOOGLE_API_KEY"] = mykeys.get_key('GOO')
os.environ["HUGGINGFACEHUB_API_TOKEN"] = mykeys.get_key('HF')

아래 링크를 복사하여 웹 브라우저에 붙여넣으세요.
https://accounts.google.com/o/oauth2/auth?client_id=35726703810-4v13dfqmilhgv6shlc3cv9i3ktuh73j1.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code


페이지에서 인증 후 받은 코드를 여기에 붙여넣으세요:  4/0AVMBsJjtoUbPZSuA90ZgRiVUXAnMOkWb155cHG3FvtZ_enDuP3X8EsMXIZ83igO047b7FA


Authentication successful.


In [31]:
from huggingface_hub import HfApi
api = HfApi()

username = "mypsyche98"
MODEL_NAME = merged_path

In [32]:
api.create_repo(
    token=mykeys.get_key('HF'),
    repo_id=f"{username}/{MODEL_NAME}",
    repo_type="model"
)

RepoUrl('https://huggingface.co/mypsyche98/Qwen3-4B-Instruct-2507_LoRA_20250818_merged', endpoint='https://huggingface.co', repo_type='model', repo_id='mypsyche98/Qwen3-4B-Instruct-2507_LoRA_20250818_merged')

In [33]:
api.upload_folder(
    token=mykeys.get_key('HF'),
    repo_id=f"{username}/{MODEL_NAME}",
    folder_path=merged_path,
)

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...LoRA_20250818_merged/tokenizer.json: 100%|##########| 11.4MB / 11.4MB            

  ...ed/model-00003-of-00006.safetensors:   1%|1         | 41.9MB / 2.99GB            

  ...ed/model-00006-of-00006.safetensors:   3%|2         | 33.4MB / 1.31GB            

  ...ed/model-00005-of-00006.safetensors:   1%|1         | 33.5MB / 2.93GB            

  ...ed/model-00001-of-00006.safetensors:   1%|          | 16.7MB / 2.97GB            

  ...ed/model-00004-of-00006.safetensors:   1%|1         | 41.9MB / 2.97GB            

  ...ed/model-00002-of-00006.safetensors:   1%|1         | 33.5MB / 2.93GB            

CommitInfo(commit_url='https://huggingface.co/mypsyche98/Qwen3-4B-Instruct-2507_LoRA_20250818_merged/commit/40c8feba2272fad80a9bc0cea99d6b6991b4b5d9', commit_message='Upload folder using huggingface_hub', commit_description='', oid='40c8feba2272fad80a9bc0cea99d6b6991b4b5d9', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mypsyche98/Qwen3-4B-Instruct-2507_LoRA_20250818_merged', endpoint='https://huggingface.co', repo_type='model', repo_id='mypsyche98/Qwen3-4B-Instruct-2507_LoRA_20250818_merged'), pr_revision=None, pr_num=None)